<img src="files/model_1_outline.png">

# change log
### Epoch = 300


sequence generator shape (corrected) (b, n, w, h, c)

In [1]:
import tensorflow as tf
import datetime
import os
import glob
import keras

In [2]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
# from patrice's blogpost
from keras_video import VideoFrameGenerator

In [5]:
classes = ['CalotTriangleDissection',
 'CleaningCoagulation',
 'ClippingCutting',
 'GallbladderDissection',
 'GallbladderPackaging',
 'GallbladderRetraction',
 'Preparation']


classes.sort()
print(classes)

['CalotTriangleDissection', 'CleaningCoagulation', 'ClippingCutting', 'GallbladderDissection', 'GallbladderPackaging', 'GallbladderRetraction', 'Preparation']


In [6]:
# some global params
SIZE = (128, 128) # height and width of frame pxl by pxl
CHANNELS = 3 # RGB or whatever
NBFRAME = 5 # num frames in sequence 
BS = 8 # Batch size

In [7]:
# pattern to get videos and classes
glob_pattern='../cholec80/model_data/{classname}/*.avi'


In [8]:
# for data augmentation
# data_aug = keras.preprocessing.image.ImageDataGenerator(
#     zoom_range=.1,
#     horizontal_flip=True,
#     rotation_range=8,
#     width_shift_range=.2,
#     height_shift_range=.2)

In [9]:
# Create video frame generator
train = VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_pattern,
    nb_frames=NBFRAME,
    split_val=.33, 
    shuffle=True,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    use_frame_cache=True)

class CalotTriangleDissection, validation count: 170, train count: 347
class CleaningCoagulation, validation count: 170, train count: 347
class ClippingCutting, validation count: 170, train count: 347
class GallbladderDissection, validation count: 170, train count: 347
class GallbladderPackaging, validation count: 170, train count: 347
class GallbladderRetraction, validation count: 170, train count: 347
class Preparation, validation count: 170, train count: 347
Total data: 7 classes for 2429 files for train


In [10]:
# getting validation data
valid = train.get_validation_generator()

Total data: 7 classes for 1190 files for validation


In [11]:
import keras_video.utils
#keras_video.utils.show_sample(train)

In [12]:
#valid.files

## BUILD CONV NET

In [13]:
from keras.layers import Conv2D, BatchNormalization, \
    MaxPool2D, GlobalMaxPool2D
from keras.applications.mobilenet import MobileNet
def build_mobilenet(shape=(224, 224, 3), nbout=3):
    model = MobileNet(
        include_top=False,
        input_shape=shape,
        weights='imagenet')
    # Keep 9 layers to train﻿﻿
    trainable = 9
    for layer in model.layers[:-trainable]:
        layer.trainable = False
    for layer in model.layers[-trainable:]:
        layer.trainable = True
        
    # adding a max pool
    output = GlobalMaxPool2D()

    return keras.Sequential([model, output])

## Build GRU

In [14]:
from keras.layers import TimeDistributed, GRU, Dense, Dropout
# Shape (5, 112, 112, 3) 5 - time sequence length 112x112 = height vs width 3 - num channels
def action_model(shape=(5, 112, 112, 3), nbout=3):
    # Create our convnet with (112, 112, 3) input shape
    convnet = build_mobilenet(shape[1:])
    
    # then create our final model
    model = keras.Sequential()
    # add the convnet with (5, 112, 112, 3) shape
    # KEY = allows you to add a time sequence to a layer one at a time
    model.add(TimeDistributed(convnet, input_shape=shape))
    # here, you can also use GRU or LSTM
    model.add(GRU(64))
    # and finally, we make a decision network
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(nbout, activation='softmax'))
    return model

## this is where you tell the model how to train - loss function, weight update mechanism

In [15]:
INSHAPE=(NBFRAME,) + SIZE + (CHANNELS,) # (5, 128, 128, 3)
print(INSHAPE)
# action model - GRU set up for Time shifted CNN
model = action_model(INSHAPE, len(classes))

#this is where you tell the model how to train - loss function, weight update mechanism
optimizer = keras.optimizers.Adam(0.001)
model.compile(
    optimizer,
    'categorical_crossentropy',
    metrics=['acc']
)

(5, 128, 128, 3)


## epochs, call backs

In [18]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

print('logs for this run are here: {}'.format(log_dir))

logs for this run are here: logs/fit/20210614-150005


In [19]:
EPOCHS=300
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
# create a "chkp" directory before to run that
# because ModelCheckpoint will write models inside
callbacks = [
    keras.callbacks.ReduceLROnPlateau(verbose=1),
    keras.callbacks.ModelCheckpoint(
        'model2_3_1_chkp/weights.{epoch:02d}-{val_loss:.2f}.hdf5',
        verbose=1),
    tensorboard_callback
]

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
model.fit(
    train,
    validation_data=valid,
    verbose=1,
    epochs=EPOCHS,
    callbacks=callbacks
)

Epoch 1/300
303/303 [==============================] - 186s 578ms/step - loss: 1.9402 - acc: 0.1700 - val_loss: 1.5337 - val_acc: 0.3361

Epoch 00001: saving model to model2_3_1_chkp\weights.01-1.53.hdf5
Epoch 2/300
303/303 [==============================] - 8s 25ms/step - loss: 1.6397 - acc: 0.2760 - val_loss: 1.5027 - val_acc: 0.3209

Epoch 00002: saving model to model2_3_1_chkp\weights.02-1.50.hdf5
Epoch 3/300
303/303 [==============================] - 7s 24ms/step - loss: 1.5495 - acc: 0.3396 - val_loss: 1.5073 - val_acc: 0.3471

Epoch 00003: saving model to model2_3_1_chkp\weights.03-1.51.hdf5
Epoch 4/300
303/303 [==============================] - 7s 24ms/step - loss: 1.5694 - acc: 0.3133 - val_loss: 1.3357 - val_acc: 0.3792

Epoch 00004: saving model to model2_3_1_chkp\weights.04-1.34.hdf5
Epoch 5/300
303/303 [==============================] - 7s 24ms/step - loss: 1.4450 - acc: 0.3576 - val_loss: 1.4102 - val_acc: 0.3758

Epoch 00005: saving model to model2_3_1_chkp\weights.05-1.


Epoch 00041: saving model to model2_3_1_chkp\weights.41-0.92.hdf5
Epoch 42/300
303/303 [==============================] - 8s 25ms/step - loss: 0.9182 - acc: 0.6387 - val_loss: 0.8835 - val_acc: 0.6588

Epoch 00042: saving model to model2_3_1_chkp\weights.42-0.88.hdf5
Epoch 43/300
303/303 [==============================] - 7s 24ms/step - loss: 0.9566 - acc: 0.6261 - val_loss: 0.8491 - val_acc: 0.6689

Epoch 00043: saving model to model2_3_1_chkp\weights.43-0.85.hdf5
Epoch 44/300
303/303 [==============================] - 7s 24ms/step - loss: 0.9039 - acc: 0.6468 - val_loss: 0.8100 - val_acc: 0.6841

Epoch 00044: saving model to model2_3_1_chkp\weights.44-0.81.hdf5
Epoch 45/300
303/303 [==============================] - 7s 24ms/step - loss: 0.8278 - acc: 0.6769 - val_loss: 0.7153 - val_acc: 0.7154

Epoch 00045: saving model to model2_3_1_chkp\weights.45-0.72.hdf5
Epoch 46/300
303/303 [==============================] - 7s 24ms/step - loss: 0.7954 - acc: 0.6829 - val_loss: 0.8182 - val_ac

Epoch 82/300
303/303 [==============================] - 7s 24ms/step - loss: 0.4863 - acc: 0.8231 - val_loss: 0.5854 - val_acc: 0.8159

Epoch 00082: saving model to model2_3_1_chkp\weights.82-0.59.hdf5
Epoch 83/300
303/303 [==============================] - 7s 24ms/step - loss: 0.5160 - acc: 0.8379 - val_loss: 0.5783 - val_acc: 0.8260

Epoch 00083: saving model to model2_3_1_chkp\weights.83-0.58.hdf5
Epoch 84/300
303/303 [==============================] - 7s 23ms/step - loss: 0.4875 - acc: 0.8345 - val_loss: 0.5661 - val_acc: 0.8345

Epoch 00084: saving model to model2_3_1_chkp\weights.84-0.57.hdf5
Epoch 85/300
303/303 [==============================] - 7s 23ms/step - loss: 0.4728 - acc: 0.8571 - val_loss: 0.5756 - val_acc: 0.8269

Epoch 00085: saving model to model2_3_1_chkp\weights.85-0.58.hdf5
Epoch 86/300
303/303 [==============================] - 7s 24ms/step - loss: 0.5378 - acc: 0.8166 - val_loss: 0.5691 - val_acc: 0.8277

Epoch 00086: saving model to model2_3_1_chkp\weights.86-

303/303 [==============================] - 7s 24ms/step - loss: 0.2583 - acc: 0.9191 - val_loss: 0.4608 - val_acc: 0.8758

Epoch 00122: saving model to model2_3_1_chkp\weights.122-0.46.hdf5
Epoch 123/300
303/303 [==============================] - 7s 24ms/step - loss: 0.2917 - acc: 0.9154 - val_loss: 0.4596 - val_acc: 0.8767

Epoch 00123: saving model to model2_3_1_chkp\weights.123-0.46.hdf5
Epoch 124/300
303/303 [==============================] - 7s 24ms/step - loss: 0.2475 - acc: 0.9152 - val_loss: 0.4693 - val_acc: 0.8716

Epoch 00124: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.

Epoch 00124: saving model to model2_3_1_chkp\weights.124-0.47.hdf5
Epoch 125/300
303/303 [==============================] - 7s 24ms/step - loss: 0.2661 - acc: 0.9152 - val_loss: 0.4591 - val_acc: 0.8784

Epoch 00125: saving model to model2_3_1_chkp\weights.125-0.46.hdf5
Epoch 126/300
303/303 [==============================] - 7s 24ms/step - loss: 0.2236 - acc: 0.9294 - val_loss: 0.458

303/303 [==============================] - 7s 24ms/step - loss: 0.2228 - acc: 0.9325 - val_loss: 0.4462 - val_acc: 0.8826

Epoch 00161: saving model to model2_3_1_chkp\weights.161-0.45.hdf5
Epoch 162/300
303/303 [==============================] - 7s 24ms/step - loss: 0.2269 - acc: 0.9287 - val_loss: 0.4442 - val_acc: 0.8843

Epoch 00162: saving model to model2_3_1_chkp\weights.162-0.44.hdf5
Epoch 163/300
303/303 [==============================] - 7s 24ms/step - loss: 0.2313 - acc: 0.9209 - val_loss: 0.4496 - val_acc: 0.8801

Epoch 00163: saving model to model2_3_1_chkp\weights.163-0.45.hdf5
Epoch 164/300
303/303 [==============================] - 7s 24ms/step - loss: 0.2619 - acc: 0.9171 - val_loss: 0.4548 - val_acc: 0.8784

Epoch 00164: saving model to model2_3_1_chkp\weights.164-0.45.hdf5
Epoch 165/300
303/303 [==============================] - 7s 24ms/step - loss: 0.2055 - acc: 0.9331 - val_loss: 0.4420 - val_acc: 0.8860

Epoch 00165: saving model to model2_3_1_chkp\weights.165-0.44

303/303 [==============================] - 7s 24ms/step - loss: 0.2189 - acc: 0.9339 - val_loss: 0.4452 - val_acc: 0.8851

Epoch 00200: saving model to model2_3_1_chkp\weights.200-0.45.hdf5
Epoch 201/300
303/303 [==============================] - 7s 24ms/step - loss: 0.2423 - acc: 0.9146 - val_loss: 0.4453 - val_acc: 0.8843

Epoch 00201: ReduceLROnPlateau reducing learning rate to 1.000000082740371e-10.

Epoch 00201: saving model to model2_3_1_chkp\weights.201-0.45.hdf5
Epoch 202/300
303/303 [==============================] - 7s 24ms/step - loss: 0.2413 - acc: 0.9244 - val_loss: 0.4497 - val_acc: 0.8868

Epoch 00202: saving model to model2_3_1_chkp\weights.202-0.45.hdf5
Epoch 203/300
303/303 [==============================] - 7s 24ms/step - loss: 0.2512 - acc: 0.9154 - val_loss: 0.4448 - val_acc: 0.8843

Epoch 00203: saving model to model2_3_1_chkp\weights.203-0.44.hdf5
Epoch 204/300
303/303 [==============================] - 7s 24ms/step - loss: 0.2264 - acc: 0.9182 - val_loss: 0.4499


Epoch 00238: saving model to model2_3_1_chkp\weights.238-0.44.hdf5
Epoch 239/300
303/303 [==============================] - 7s 24ms/step - loss: 0.2598 - acc: 0.9245 - val_loss: 0.4441 - val_acc: 0.8834

Epoch 00239: saving model to model2_3_1_chkp\weights.239-0.44.hdf5
Epoch 240/300
303/303 [==============================] - 7s 23ms/step - loss: 0.2444 - acc: 0.9266 - val_loss: 0.4457 - val_acc: 0.8843

Epoch 00240: saving model to model2_3_1_chkp\weights.240-0.45.hdf5
Epoch 241/300
303/303 [==============================] - 7s 24ms/step - loss: 0.2535 - acc: 0.9250 - val_loss: 0.4557 - val_acc: 0.8818

Epoch 00241: ReduceLROnPlateau reducing learning rate to 1.0000001179769417e-14.

Epoch 00241: saving model to model2_3_1_chkp\weights.241-0.46.hdf5
Epoch 242/300
303/303 [==============================] - 7s 24ms/step - loss: 0.2753 - acc: 0.9180 - val_loss: 0.4426 - val_acc: 0.8843

Epoch 00242: saving model to model2_3_1_chkp\weights.242-0.44.hdf5
Epoch 243/300
303/303 [===========


Epoch 00277: saving model to model2_3_1_chkp\weights.277-0.44.hdf5
Epoch 278/300
303/303 [==============================] - 7s 24ms/step - loss: 0.2594 - acc: 0.9232 - val_loss: 0.4422 - val_acc: 0.8834

Epoch 00278: saving model to model2_3_1_chkp\weights.278-0.44.hdf5
Epoch 279/300
303/303 [==============================] - 7s 24ms/step - loss: 0.2449 - acc: 0.9303 - val_loss: 0.4447 - val_acc: 0.8834

Epoch 00279: saving model to model2_3_1_chkp\weights.279-0.44.hdf5
Epoch 280/300
303/303 [==============================] - 7s 23ms/step - loss: 0.2311 - acc: 0.9281 - val_loss: 0.4452 - val_acc: 0.8818

Epoch 00280: saving model to model2_3_1_chkp\weights.280-0.45.hdf5
Epoch 281/300
303/303 [==============================] - 7s 24ms/step - loss: 0.2472 - acc: 0.9298 - val_loss: 0.4473 - val_acc: 0.8775

Epoch 00281: ReduceLROnPlateau reducing learning rate to 1.0000000830368326e-17.

Epoch 00281: saving model to model2_3_1_chkp\weights.281-0.45.hdf5
Epoch 282/300
303/303 [===========